**AIM:** To forecast the total amount of products sold in every shop for the test set given with daily historical sales data using basic ensembling techniques., note that the list of shops and products slightly changes every month.



**IMPLEMENTATION:**

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import os
from datetime import datetime

# Import specific packages
import re
from collections import Counter
from scipy.sparse import csr_matrix
from itertools import compress
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
item_cat = pd.read_csv('./data/item_categories.csv')
items = pd.read_csv('./data/items.csv')
sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')
shops = pd.read_csv('./data/shops.csv')

NameError: ignored

# Data wrangling

**Merge and flatten dataframe**

First of all, let's join the `items` and `item_cat` together.

In [ ]:
items = pd.merge(items, item_cat, how='left', on=['item_category_id'])

**Apply tfidf transform on the `item_name`, `item_category_name` and `shop_name`**

Let convert a collection of raw documents to a matrix of TF-IDF features using the class `sklearn.feature_extraction.text.TfidfVectorizer`, which is equivalent to `CountVectorizer` followed by `TfidfTransformer`.

In [ ]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_name = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())

Let's create a function to merge the items and item_name together.

In [ ]:
def merge_dataframe(df_left, df_right, column_name_prefix):
    for column in df_right.columns.values:
        col = column_name_prefix + str(column)
        df_left[col] = df_right[column]

In [ ]:
merge_dataframe(items, item_name, 'item_name')

Let's do the same transformation to the item_category_name and shop_name

In [ ]:
feature_cnt = 25
tfidf = TfidfVectorizer(max_features=feature_cnt)
item_cat_name = pd.DataFrame(tfidf.fit_transform(items['item_category_name']).toarray())

merge_dataframe(items, item_cat_name, 'item_cat_name')

tfidf = TfidfVectorizer(max_features=feature_cnt)
shop_name = pd.DataFrame(tfidf.fit_transform(shops['shop_name']).toarray())

merge_dataframe(shops, shop_name, 'shop_name')

As we notice that there are some negative item_price and negative item_cnt_day. We will filter out those records.

In [ ]:
sales_train = sales_train[(sales_train['item_price']>0) & (sales_train['item_cnt_day']>0)]

**Generate table item_price_latest with the last item_price**

In [ ]:
item_price_latest = sales_train.sort_values(by=['date'], ascending=False).groupby(['item_id', 'shop_id'], as_index=False)['item_price'].first()#.rename(columns={'item_price': 'item_price'})

,item_id,shop_id,item_price
0,0,54,58.0
1,1,55,4490.0
2,2,54,58.0
3,3,54,58.0
4,4,54,58.0


**Extract date columns to year, month, day ...**

Now let's extract the `date` column to: `year`, `month`, `day`, `day_of_year`, `weekday`.

In [ ]:
sales_train['date'] = sales_train['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y'))
sales_train['year'] = sales_train['date'].apply(lambda x: x.year)
sales_train['month'] = sales_train['date'].apply(lambda x: x.month)

**Aggregate on month level**

In [ ]:
sales_train1 = sales_train.groupby(['shop_id', 'date_block_num', 'item_id', 'year', 'month'], as_index=False)['item_cnt_day'].sum().rename(columns={'item_cnt_day':'item_cnt_month'})

In [ ]:
sales_train1.describe()

,shop_id,date_block_num,item_id,year,month,item_cnt_month
count,1.608226e+06,1.608226e+06,1.608226e+06,1.608226e+06,1.608226e+06,1.608226e+06
mean,3.280798e+01,1.466502e+01,1.068148e+04,2.013793e+03,6.154981e+00,2.273154e+00
std,1.653645e+01,9.542384e+00,6.238856e+03,7.773325e-01,3.455207e+00,8.653236e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,2.013000e+03,1.000000e+00,1.000000e+00
25%,2.100000e+01,6.000000e+00,5.046000e+03,2.013000e+03,3.000000e+00,1.000000e+00
50%,3.100000e+01,1.400000e+01,1.049700e+04,2.014000e+03,6.000000e+00,1.000000e+00
75%,4.700000e+01,2.300000e+01,1.606000e+04,2.014000e+03,9.000000e+00,2.000000e+00
max,5.900000e+01,3.300000e+01,2.216900e+04,2.015000e+03,1.200000e+01,2.253000e+03


The model can be influenced by the outliers. Let's use apply clip to the outliers here, where `item_cnt_month` > (mean + 2std) = 20. 

In [ ]:
sales_train1['item_cnt_month'] = sales_train1['item_cnt_month'].clip(0, 20)

** Create feature `item_cnt_prev_month`**

In [ ]:
# sort sales_train1 on ['date_block_num']
sales_train1 = sales_train1.sort_values('date_block_num')
sales_train1['item_cnt_prev_month'] = sales_train1.groupby(['shop_id', 'item_id'])['item_cnt_month'].shift(1).fillna(0)

In [ ]:
# for the testing, the item_cnt_prev_month is the last month on the sales_train1
test_item_cnt_prev_month = sales_train1[['item_id', 'shop_id', 'item_cnt_month', 'date_block_num']].sort_values('date_block_num', ascending=False).groupby(['item_id', 'shop_id'], as_index=False).first()
test_item_cnt_prev_month = test_item_cnt_prev_month[['item_id', 'shop_id', 'item_cnt_month']].rename(columns={'item_cnt_month': 'item_cnt_prev_month'})


**Create feature `item_cnt_month_mean`**

`item_cnt_month_mean`: average of item_cnt_month per shop

In [ ]:
# Exclude the date_block_num ==33
item_month_mean = sales_train1[sales_train1['date_block_num']!=33].groupby(['item_id', 'shop_id'], as_index=False)['item_cnt_month'].mean().rename(columns={'item_cnt_month': 'item_cnt_month_mean'})
sales_train1 = sales_train1.merge(item_month_mean, how='left', on=['item_id', 'shop_id'])
sales_train1.head()

,shop_id,date_block_num,item_id,year,month,item_cnt_month,item_cnt_prev_month,item_cnt_month_mean
0,0,0,32,2013,1,6.0,0.0,8.0
1,37,0,18636,2013,1,1.0,0.0,1.0
2,37,0,18610,2013,1,1.0,0.0,1.0
3,37,0,18581,2013,1,1.0,0.0,1.0
4,37,0,18580,2013,1,1.0,0.0,1.0


In [ ]:
# For test data, we can actually use the date_block_num ==33
test_item_month_mean = sales_train1.groupby(['item_id', 'shop_id'], as_index=False)['item_cnt_month'].mean().rename(columns={'item_cnt_month': 'item_cnt_month_mean'})
test_item_month_mean.head()

,item_id,shop_id,item_cnt_month_mean
0,0,54,1.0
1,1,55,1.2
2,2,54,1.0
3,3,54,1.0
4,4,54,1.0


**Generate table item_price_avg for each shop and month**

As we need to predict the sales on month level, let's also aggregate the item_price on month level.

In [ ]:
item_price_avg = sales_train.groupby(['item_id', 'shop_id', 'year', 'month'], as_index=False)['item_price'].mean()
item_price_avg.head()

,item_id,shop_id,year,month,item_price
0,0,54,2014,9,58.0
1,1,55,2014,4,4490.0
2,1,55,2014,7,4490.0
3,1,55,2014,8,4490.0
4,1,55,2014,9,4490.0


**Create training data**

In [ ]:
sales_train2 = pd.merge(sales_train1, item_price_avg, how='left', on=['shop_id','item_id','year','month'])
sales_train3 = pd.merge(sales_train2, items, how='left', on=['item_id'])
sales_train4 = pd.merge(sales_train3, shops, how='left', on=['shop_id'])
train = sales_train4

**Create test data**

In [ ]:
test2 = pd.merge(test, item_price_latest, how='left', on=['shop_id','item_id'])
test3 = pd.merge(test2, items, how='left', on=['item_id'])
test4 = pd.merge(test3, shops, how='left', on=['shop_id'])
test5 = pd.merge(test4, test_item_cnt_prev_month[['item_id', 'shop_id', 'item_cnt_prev_month']], how='left', on=['item_id', 'shop_id'])
test6 = pd.merge(test5, test_item_month_mean, how='left', on=['item_id', 'shop_id'])
df_test = test6

**Label encoding**

In [ ]:
for col in ['shop_id', 'item_id', 'item_category_id']:
    train[col] = train[col].astype(str)
    df_test[col] = df_test[col].astype(str)

let's add the month and year to the df_test as well.

In [ ]:
train.sort_values(by=['year','month'], ascending=[False, False]).head(1)

,shop_id,date_block_num,item_id,year,month,item_cnt_month,item_cnt_prev_month,item_cnt_month_mean,item_price,item_name,...,shop_name15,shop_name16,shop_name17,shop_name18,shop_name19,shop_name20,shop_name21,shop_name22,shop_name23,shop_name24
1576705,58,33,2252,2015,10,4.0,1.0,2.259259,399.0,"Call Of Duty: Modern Warfare 3 [PC, Jewel]",...,0.0,0.0,0.0,0.0,0.0,0.0,0.322815,0.0,0.689588,0.648274


In [ ]:
df_test['year'] = 2015
df_test['month'] = 11
df_test['date_block_num'] = 34

In [ ]:
train = shuffle(train, random_state=42)

In [ ]:
X = train[[col for col in train.columns.values if col not in ['item_name', 'item_category_name', 'shop_name', 'item_cnt_month', 'item_cnt_prev_month', 'item_cnt_month_mean']]].fillna(0)

y = train['item_cnt_month'].fillna(0)

list_training = list(X['date_block_num']<33)
list_testing = list(X['date_block_num']==33)

X_train2 = X[X['date_block_num']<33]
y_train2 = y[list_training].fillna(0)
X_test2 = X[X['date_block_num']==33]
y_test2 = y[list_testing].fillna(0)

In [ ]:
reg = ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=42)
reg.fit(X_train2, y_train2)

y_train_pred = reg.predict(X_train2)
y_test_pred = reg.predict(X_test2)

rmse_train = np.sqrt(mean_squared_error(y_train2, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test2, y_test_pred))

print('rmse_train:', rmse_train)
print('rmse_test:', rmse_test)

rmse_train: 1.9700191091132222
rmse_test: 2.154155708321535


Interstingly, the model works better without the feature `item_cnt_prev_month` & `item_cnt_month_mean`.

In [ ]:
rf_reg2 = RandomForestRegressor(n_estimators=25, max_depth=10, random_state=42)
rf_reg2.fit(X_train2, y_train2)

y_train2_pred = rf_reg2.predict(X_train2)
y_test2_pred = rf_reg2.predict(X_test2)

In [ ]:
rmse_train2 = np.sqrt(mean_squared_error(y_train2, y_train2_pred))
rmse_test2 = np.sqrt(mean_squared_error(y_test2, y_test2_pred))

print('rmse_train:', rmse_train2)
print('rmse_test:', rmse_test2)

rmse_train: 1.4905336648206977
rmse_test: 2.43961176662877


In [ ]:
df_test2 = df_test[[col for col in X_train2.columns.values]].fillna(0)
test_pred = reg.predict(df_test2)

In [ ]:
test_pred = np.round(test_pred,1)

In [ ]:
test['item_cnt_month'] = test_pred

In [ ]:
test.to_csv(path_or_buf='./data/result.csv', columns=['ID', 'item_cnt_month'], index=False)